In [6]:
import numpy as np
from math import exp, log, sqrt
import scipy.stats as ss
import matplotlib.pyplot as plt

def longstaff_schwartz_lsm_american_put(paths, r, K, dt):

    # periods e' o numero de passos simulados, incluindo o inicial e runs
    # e' o numero de simulacoes realizadas
    periods = paths.shape[1]
    runs = paths.shape[0]

    cashflows = np.matrix(np.zeros(paths.shape))
    stops = np.matrix(np.zeros(paths.shape), dtype = bool)

    # Trata o ultimo perido, no qual a opcao sempre e' exercida
    cashflows[:, periods - 1] = np.maximum(K - paths[:, periods - 1], 0)

    # Tratamento dos periodos intermediarios
    for period in reversed(range(1, periods - 1)):

        # Precos simulados
        prices = np.asarray(paths[:, period].T)[0]

        # Verifica quais caminhos estao on the money no instante avaliado
        runs_on_the_money = np.where(prices < K)[0]

        # Para cada caminho on the money, devemos comparar o resultado do
        # exercicio com o valor esperado de esperar. Para calcular o valor
        # esperado de esperar, iremos fazer uma regressao dos retornos
        # futuros dos caminhos que estão on the money em funcao dos prceos
        # atuais

        # Resultado do exercicio imediato dos caminhos on the money
        immed_exerc = K - prices[runs_on_the_money]

        # Vetor com os coeficientes de desconto dos fluxos futuros
        discount = np.matrix([1 / exp(r * t * dt) for t in range(1, periods - period)])

        # Retornos obtidos no futuro dos caminhos on the money
        future_cashflows = np.asarray(discount * cashflows[runs_on_the_money, period + 1:].T)[0]

        # Regressao nos caminhos que estão on the money
        coefs = np.polyfit(prices[runs_on_the_money], future_cashflows, 2)
        poly = np.poly1d(coefs)

        # Calcula os retornos esperados se nao vender
        expected_delayed_exerc = poly(prices[runs_on_the_money])

        # Caminhos onde e' melhor exercer imediatamente
        exercise = runs_on_the_money[immed_exerc > expected_delayed_exerc]
        stops[exercise, period] = True
        stops[exercise, period + 1:] = False

        # Atualiza os retornos dos caminhos onde e' melhor exercer e zera os
        # retornos futuros desses caminhos
        cashflows[exercise, period] = (K - prices)[exercise]
        cashflows[exercise, period + 1:] = 0


    # Calcula o valor presente medio dos varios caminhos
    discount = np.matrix([1 / exp(r * t * dt) for t in range(0, periods)])
    option_value = np.mean(cashflows * discount.T)

    return(option_value)

In [4]:
def test_longstaff_schwartz():

    # Exemplo do paper / aula
    paths = np.matrix([[1.00, 1.09, 1.08, 1.34],
                       [1.00, 1.16, 1.26, 1.54],
                       [1.00, 1.22, 1.07, 1.03],
                       [1.00, 0.93, 0.97, 0.92],
                       [1.00, 1.11, 1.56, 1.52],
                       [1.00, 0.76, 0.77, 0.90],
                       [1.00, 0.92, 0.84, 1.01],
                       [1.00, 0.88, 1.22, 1.34]])

    r = 0.06
    K = 1.1
    d = 1

    return(longstaff_schwartz_lsm_american_put(paths, r, K, d))

In [5]:
test_longstaff_schwartz()

0.11443433004505696